# Теория сложных сетей в экономике

# Лекция 4: Выявление сообществ в сетях

## Часть 3. Методы расстановки меток (label propagation)

## Макрушин Сергей Вячеславович, Финансовый университет, 2020 г.
v 0.2

__Задача расстановки меток__

* множество <em class="cb">меток узлов</em> (node labels): $\mathcal{Y}$ (в разных задачах: бинарные, категориальные, ординальные или числовые)
    * количество различных меток: $|\mathcal{Y}|=l$
* $y_{ic}$ - вероятность того, что узел $i \in \mathcal{N}$ имеет метку $c \in \mathcal{Y}$
    * $\sum_c y_{ic}=1$
    * <em class="hn"></em> матрица $\mathbf{Y}=y_{ic}$ размерностью $n \times l$ описывает распределение узлов по меткам (классам)

      
* пусть изначально расстановка меток известна только для $l$ узлов (обычно $l \ll n$)    
    * узлы, изначально имеющие метки, относятся к множеству: $\mathcal{N}_l$
        * обзначим $\mathcal{N}_u$ множество узлов не имеющих меток $\mathcal{N}_u=\mathcal{N} \setminus \mathcal{N}_l$
    * каждый узел $n_i \in \mathcal{N}_l$ имеет только одну изначальную метку:  $y_{ic} = 1 \wedge y_{ij} = 0, j \ne p $ 
    * для удобства будем считать, что метки изначально известны для узлов, имеющих первые $l$ индексов, соответственно: 
        * $i \le l: n_i \in \mathcal{N}_l$
        * $i > l: n_i \in \mathcal{N}_u$
    * <em class="hn"></em> матрица $\mathbf{Y_l}$ размерностью $n \times l$ содержит изначальную расстановку меток (в каждой строке матрицы 
        * в каждой строке матрицы $\mathbf{Y_l}$ одна единица, остальные значения - нули
    * Матрица расстановки меток в начальном состоянии имеет вид: $$\mathbf{Y_0} = \begin{pmatrix}
\mathbf{Y_{l}}\\
\mathbf{Y_{u}}\\
\end{pmatrix}= \begin{pmatrix}
\mathbf{Y_{l}}\\
\mathbf{0}\\
\end{pmatrix}$$

<center> 
__Пример задачи расстановки меток__

<img src="clust3_3.png" alt="Пример задачи расстановки меток" style="width: 400px;"/>
</center>

* <em class="cr">__Допущение__</em>: связи сети отражают корелляцию между значениями меток
* <em class="ex"></em> В социальных сетях связанные узлы часто имеют идентичние (или близкие) признаки. Причина:
    * гомоифилия (homophily): схожесть узлов (одниковые признаки) приводит к орбазованию связей между ними
    * влияние (inﬂuence): наличие связей приводит к распространению свойст вмежду ними


* <em class="df"></em> __Задача расстановки меток__ (label propagation problem): по изначальной расстоновке меток $\mathbf{Y_l}$ в сети с матрицей инцедентности $\mathbf{A}$ (или, в общем случае, со взвешенными ориентированными связями, определяемыми матрицей $\mathbf{W}$) однозначно определить $\tilde{\mathbf{Y}}$ - значения меток для всех узлов, включая узлы  из $\mathcal{N}_u$

* Для решения задачи необходимо, чтобы в сети была обеспечена <em class="cb">связность с метками</em>, т.е. из каждого узла сети можно было построить путь хотя бы до одного из изначально помеченных узлов

* Данную задачу можно рассматривать как задачу <em class="cb">обучения с частичным привлечением учителя</em> (semi-supervised learning) - разновидность обучения с учителем, которое также использует неразмеченные данные для тренировки. Обычно небольшое количество размеченных данных и большое количество неразмеченных данных. Данный подход занимает промежуточную позицию между обучением без учителя (без привлечения каких-либо размеченных данных для обучения) и обучением с учителем (с использованием в обучении лишь размеченных данных).

__Случайное блуждение в сети__

* <em class="df"></em> __Случайное блуждение__ (random walk) - случайный процесс перехода между состояниями (в нашем случае - узлами) определяемый матрицей перехода $\mathbf{P}$, в которой фиксируется вероятность случайного перехода $p_{ij}$ из узла $n_i$ в узел $n_j$. Для корректности вероятностного определения должны выполняться следующие требования:
    * $0 \le p_{ij} \le 1$
    * $\sum_j p_{ij}=1$
* Матрица переходов получается нормализацей матрицы смежности: $\mathbf{P}=\mathbf{D}^{-1}\mathbf{A}$, где $\mathbf{D}=\text{diag}(k_{i})$, $k_i = \sum_j a_{ij}$ (определение подходит и для взвешенных и ориентированных связей $\mathbf{A}=\mathbf{W}$)

<center> 
__Пример случайного блуждания в сети__

<img src="clust3_4.png" alt="Пример случайного блуждания в сети" style="width: 400px;"/>
</center>

* вектор $\mathbf{p}^t=p^t_i$ определяет вероятность нахождения в узле $i$ во время шага $t$ 
    * это вектор распределения вероятности: $\sum_i p^t_i = 1$
* шаг случайного блуждания можно записать как: $\mathbf{p}^{t+1}=\mathbf{P} \mathbf{p}^t$  
* <em class="hn"></em> переход из начального состояния в состояние на шаге $t$ = $\mathbf{p}^{t}=\mathbf{P}^t \mathbf{p}^0$  
* в пределе получим стационарное распределение по узлам сети $\mathbf{p}^{\infty}$: $\mathbf{p}^{\infty}=\lim_{t \to \infty} \mathbf{P}^t \mathbf{p}^0 = \mathbf{P}^{\infty} \mathbf{p}^0$
    * для стационарного распределения верно: $\mathbf{P}^{\infty}=\mathbf{P}\mathbf{P}^{\infty}$
    * если $\mathbf{p}^0=\mathbf{e}_i$, то стационарное распределение будет i-м столбцом из матрицы $\mathbf{P}^{\infty}$: $\mathbf{p}^{\infty}_i=\{p_{1i},p_{2i},\ldots, p_{ni}\}$


-----
(больше про стационарное состояние:  http://www.leonidzhukov.net/hse/2016/networks/lectures/lecture11.pdf)

* в пределе получим стационарное распределение по узлам сети $\mathbf{p}^{\infty}$: $\lim_{t \to \infty} \mathbf{P}^t \mathbf{p}^0 = \mathbf{p}^{\infty}$
    * для стационарного распределения верно: $\mathbf{p}^{\infty}=\mathbf{P}\mathbf{p}^{\infty}$

__Расстановка меток на основе случайного блуждания__

<em class="cr">__Идея__</em>: расстановка меток на основе случайного блуждания:
* процесс случайного блуждания (специализированный!) определен для рассматриваемой сети

* вероятность отнесения метки $l$ к узлу $n_i$ равна веротности достижения узлов с меткой $l$ в процессе случайного блуждания, начатого из узла $n_i$ при количестве шагов $t \to \infty$ 

* т.е.: $\tilde{y}_{ic}=\sum_{c: y^l_{ic}=1} p_{ij}^{\infty}y^l_{ic}$, где $y^l_{ic}$ элементы матрицы $\mathbf{Y}_l$, определяющей исходную расстановку меток

* используя матрицу расстановки меток в начальном состоянии $\mathbf{Y_0}$ и матрицу перехода к стационарному состоянияю $\mathbf{P}^{\infty}$ можно получить распеределение по меткам для всех узлов: $\tilde{\mathbf{Y}}$: $\tilde{\mathbf{Y}}=\mathbf{P}^{\infty}\mathbf{Y_0}$

* если на выходе требуется указывать для каждого узла одну метку, то ее можно определить следующим образом: $\hat{y}_{ic}= \arg \underset{c \in \mathcal{Y}}{\max} \tilde{y}_{ic}$ 

* процесс случайного блуждания нужно адаптировать:
    * изначально помеченные узлы не должны менять принадлежность
    * если в процессе случайного блуждения из $n_i$ мы достигли изначально помеченного узла, то продолжать эту траекторию больше нет смылса
    * <em class="hn"></em> процесс случайного блуждания с ловушками        

__Процесс случайного блуждания с ловушками__

* <em class="df"></em>__Процесс случайного блуждания с ловушками__: определим что в матрице перехода $\mathbf{P}$ для случайных блажданий есть <em class="cb">узлы-ловушки</em> (traps, absorbing states): попав в них при блуждании мы остаемся в них (не можем их поикнуть) во время всех последующих итераций
    * т.е. для узла-ловушки $n_i$: $p_{ii}=1$ и $i \ne j: p_{ij}=0$    
    * если в качестве узлов-ловушек выбрать узлы $n_i \in \mathcal{N}_l$, то:
        * состояние узлов-ловушек всегда остается неизменным
        * траектории случайных блажданий из узлов, относящихся к $\mathcal{N}_u$, будут завершаться на изначально помеченных узлах
* Учитывая, что узлы с изначально известными метками имеют индексы $i \le l$, матрица переходов для случайного блуждания с ловушками будет иметь блочный вид:
$$\mathbf{P} = \begin{pmatrix}
\mathbf{P}_{ll}& \mathbf{P}_{lu}\\
\mathbf{P}_{ul}& \mathbf{P}_{uu}\\
\end{pmatrix} = \begin{pmatrix}
\mathbf{I}& \mathbf{0}\\
\mathbf{P}_{ul}& \mathbf{P}_{uu}\\
\end{pmatrix}$$
* рассчитывая $\lim_{t \to \infty} \mathbf{P}^t$ мы получим:
$$\mathbf{P} = \begin{pmatrix}
\mathbf{I}& \mathbf{0}\\
\mathbf{P}_{ul}& \mathbf{P}_{uu}\\
\end{pmatrix} = \begin{pmatrix}
\mathbf{I}& \mathbf{0}\\
(\mathbf{I}-\mathbf{P}_{uu})^{-1}\mathbf{P}_{ul}& \mathbf{P}^{\infty}_{uu}\\
\end{pmatrix}$$
* т.е. $\tilde{\mathbf{Y}}_{u}=(\mathbf{I}-\mathbf{P}_{uu})^{-1}\mathbf{P}_{ul}\mathbf{Y}_{l}$


__Алгоритмы расстановки меток с использованием случайного блуждания__

<center> 
__Базовый вариант АРМ__

<img src="clust3_1.png" alt="Базовый вариант АРМ" style="width: 500px;"/>
</center>

<center> 
__Вариант разивтия АРМ__

<img src="clust3_2.png" alt="Базовый вариант АРМ" style="width: 500px;"/>
</center>

# Конец Части 3.
-----

Далее идет техническая информация.

In [1]:
%%html
<style>


b.n {
    font-weight: normal;        
}

b.grbg {
    background-color: #a0a0a0;      
}

b.r {
    color: #ff0000;    
}


b.b {    
    color: #0000ff;    
}

b.g {
    color: #00ff00;    
}


// add your CSS styling here

list-style: none;

ul.s {
//    list-style-type: none;
    list-style: none;
//    background-color: #ff0000;  
//    color: #ffff00;
//  padding-left: 1.2em;
//  text-indent: -1.2em;
}

li.t {
    list-style: none;
//  padding-left: 1.2em;
//  text-indent: -1.2em;    
}


*.r {
    color: #ff0000;    
}

li.t:before {
    content: "\21D2";    
//    content: "►";
//    padding-left: -1.2em;    
    text-indent: -1.2em;    
    display: block;
    float: left;
    
    
//    width: 1.2em;
//    color: #ff0000;
}

i.m:before {
    font-style: normal;    
    content: "\21D2";  
}
i.m {
    font-style: normal; 
}    

/*--------------------*/
/* em {
    font-style: normal; 
} */


em.bl {
    font-style: normal;     
    font-weight: bold;        
}

/* em.grbg {
    font-style: normal;         
    background-color: #a0a0a0;      
} */

em.cr {
    font-style: normal;         
    color: #ff0000;    
}

em.cb {    
    font-style: normal;         
    color: #0000ff;    
}

em.cg {
    font-style: normal;         
    color: #00ff00;    
}

/*--------------------*/

em.qs {
    font-style: normal; 
}

em.qs::before {
    font-weight: bold;    
    color: #ff0000;    
    content: "Q:";  
}

em.an {
    font-style: normal; 
}

em.an:before {
    font-weight: bold;    
    color: #0000ff;    
    content: "A:";  
}
    
em.ex {
    font-style: normal; 
}

em.ex:before {
    font-weight: bold;    
    color: #00ff00;    
    content: "Ex:";  
} 
    
em.df {
    font-style: normal; 
}

em.df:before {
    font-weight: bold;    
    color: #000000;    
    content: "Def:";  
}    

em.pl {
    font-style: normal; 
}

em.pl:before {
    font-weight: bold;    
    color: #0000ff;    
    content: "+";  
}    

em.mn {
    font-style: normal; 
}

em.mn:before {
    font-weight: bold;    
    color: #0000ff;    
    content: "-";  
}        

em.plmn {
    font-style: normal; 
}

em.plmn:before {
    font-weight: bold;    
    color: #0000ff;    
    content: "\00B1";\\"&plusmn;";  
}
    
em.hn {
    font-style: normal; 
}

em.hn:before {
    font-weight: bold;    
    color: #0000ff;    
    content: "\21D2";\\"&rArr;";  
}     
    
</style>